# GPU-Accelerated Feature Extraction and Cross-Attention CNN Training

## Cross-Attention CNN Personality Trait Prediction Project

This notebook performs comprehensive feature extraction and model training with GPU acceleration and production-ready features:

### Pipeline Overview:
1. **GPU Configuration** - Optimize TensorFlow GPU settings and memory management
2. **Data Analysis** - Analyze preprocessed data structure (68K+ faces, 81K+ optical flows)
3. **Static Feature Extraction** - Extract 512-dim features using ResNet-50 with GPU batching
4. **Dynamic Feature Extraction** - Extract 256-dim features using I3D with GPU optimization
5. **Data Alignment & Validation** - Ensure proper video-to-label mapping across 960 samples
6. **Cross-Attention Training** - Train complete model with monitoring and checkpointing
7. **Model Evaluation** - Comprehensive evaluation with visualizations

### Data Structure:
- **Input**: `data/processed/faces/` (68,459 faces) and `data/processed/optical_flow/` (81,330 flows)
- **Labels**: ChaLearn dataset OCEAN personality traits
- **Output**: Extracted features, trained Cross-Attention CNN model, and evaluation results

### Key Features:
- **GPU Acceleration**: Optimized batch processing for large-scale feature extraction
- **Resume Functionality**: Smart checkpointing to resume from interruptions
- **Memory Management**: Efficient handling of large datasets
- **Progress Tracking**: Comprehensive logging and progress monitoring
- **Error Recovery**: Robust error handling and data validation

In [10]:
# Import Required Libraries
import tensorflow as tf
import cv2
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import time
from datetime import datetime
import json
from tqdm.notebook import tqdm
import warnings
import sys
import yaml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import h5py
from concurrent.futures import ThreadPoolExecutor
import gc

warnings.filterwarnings('ignore')

# Add project root to path for imports
project_root = os.path.abspath('.')
sys.path.insert(0, project_root)

print(f"🔧 Environment Setup")
print(f"   TensorFlow version: {tf.__version__}")
print(f"   NumPy version: {np.__version__}")
print(f"   OpenCV version: {cv2.__version__}")
print(f"   Project root: {project_root}")

ImportError: numpy._core.multiarray failed to import

In [ ]:
# GPU Configuration and Memory Management
print("🔍 GPU Configuration and Optimization")
print("=" * 60)

# List and configure physical GPU devices
physical_gpus = tf.config.experimental.list_physical_devices('GPU')
logical_gpus = tf.config.experimental.list_logical_devices('GPU')

print(f"Physical GPUs: {len(physical_gpus)}")
print(f"Logical GPUs: {len(logical_gpus)}")

if physical_gpus:
    print("✅ GPU Support Available!")
    for i, gpu in enumerate(physical_gpus):
        print(f"   GPU {i}: {gpu}")
    
    try:
        # Enable memory growth to prevent TensorFlow from allocating all GPU memory
        for gpu in physical_gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        
        # Set memory limit if needed (optional)
        # tf.config.experimental.set_memory_limit(physical_gpus[0], 8192)  # 8GB limit
        
        print("✅ GPU memory growth enabled")
        
        # Configure GPU execution
        tf.config.experimental.set_synchronous_execution(False)
        print("✅ Asynchronous execution enabled")
        
    except RuntimeError as e:
        print(f"⚠️ GPU configuration error: {e}")
        
else:
    print("❌ No GPU detected - using CPU (will be slower)")

# Test GPU with matrix operations
print(f"\n🧪 GPU Performance Test:")
with tf.device('/GPU:0' if physical_gpus else '/CPU:0'):
    # Test tensor operations
    start_time = time.time()
    a = tf.random.normal((1000, 1000))
    b = tf.random.normal((1000, 1000))
    c = tf.matmul(a, b)
    end_time = time.time()
    
    print(f"   Matrix multiplication (1000x1000): {(end_time - start_time)*1000:.2f}ms")
    print(f"   Device used: {c.device}")
    print(f"   Result shape: {c.shape}")

# Memory cleanup
del a, b, c
gc.collect()

In [ ]:
# Data Structure Analysis and Validation
print("📊 Data Structure Analysis and Validation")
print("=" * 60)

# Load preprocessing results
try:
    with open('results/complete_preprocessing_results.json', 'r') as f:
        preprocessing_results = json.load(f)
    print("✅ Loaded preprocessing results")
except FileNotFoundError:
    print("❌ Preprocessing results not found - run preprocessing pipeline first")
    preprocessing_results = {}

# Directory configuration
faces_base_dir = 'data/processed/faces'
flow_base_dir = 'data/processed/optical_flow'
results_dir = 'results'
features_dir = os.path.join(results_dir, 'features')
models_dir = os.path.join(results_dir, 'models')

# Create output directories
Path(features_dir).mkdir(parents=True, exist_ok=True)
Path(models_dir).mkdir(parents=True, exist_ok=True)

# Analyze current data structure
data_stats = {
    'faces_directory_exists': os.path.exists(faces_base_dir),
    'optical_flow_directory_exists': os.path.exists(flow_base_dir),
    'total_face_images': 0,
    'total_optical_flow_files': 0,
    'training_directories': {},
    'video_count_per_directory': {},
    'analysis_timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
}

if data_stats['faces_directory_exists']:
    print(f"📁 Analyzing faces directory: {faces_base_dir}")
    
    training_dirs = sorted([d for d in os.listdir(faces_base_dir) 
                           if os.path.isdir(os.path.join(faces_base_dir, d))])
    
    for training_dir in tqdm(training_dirs, desc="Analyzing directories"):
        training_path = os.path.join(faces_base_dir, training_dir)
        video_dirs = [d for d in os.listdir(training_path) 
                     if os.path.isdir(os.path.join(training_path, d))]
        
        dir_face_count = 0
        for video_dir in video_dirs:
            video_path = os.path.join(training_path, video_dir)
            face_files = [f for f in os.listdir(video_path) if f.endswith('.jpg')]
            dir_face_count += len(face_files)
        
        data_stats['training_directories'][training_dir] = {
            'videos': len(video_dirs),
            'faces': dir_face_count
        }
        data_stats['video_count_per_directory'][training_dir] = len(video_dirs)
        data_stats['total_face_images'] += dir_face_count

if data_stats['optical_flow_directory_exists']:
    print(f"📁 Analyzing optical flow directory: {flow_base_dir}")
    
    for root, dirs, files in os.walk(flow_base_dir):
        data_stats['total_optical_flow_files'] += len([f for f in files if f.endswith('.npy')])

# Display analysis results
print(f"\n📈 Data Analysis Results:")
print(f"   • Faces directory: {'✅' if data_stats['faces_directory_exists'] else '❌'}")
print(f"   • Optical flow directory: {'✅' if data_stats['optical_flow_directory_exists'] else '❌'}")
print(f"   • Total face images: {data_stats['total_face_images']:,}")
print(f"   • Total optical flow files: {data_stats['total_optical_flow_files']:,}")
print(f"   • Training directories: {len(data_stats['training_directories'])}")

if data_stats['training_directories']:
    print(f"\n📋 Directory breakdown:")
    for dir_name, stats in data_stats['training_directories'].items():
        print(f"   {dir_name}: {stats['videos']} videos, {stats['faces']:,} faces")

# Save analysis results
with open(os.path.join(results_dir, 'data_analysis_results.json'), 'w') as f:
    json.dump(data_stats, f, indent=2)

print(f"\n💾 Analysis results saved to: {os.path.join(results_dir, 'data_analysis_results.json')}")

In [ ]:
# Import and Initialize Model Components
print("🤖 Initializing Model Components")
print("=" * 50)

try:
    # Import existing model components
    from models.static_feature_extractor.feature_extractor import StaticFeatureExtractor
    from models.dynamic_feature_extractor.feature_extractor import DynamicFeatureExtractor
    from src.models.cross_attention import CrossAttention
    from src.models.personality_model import CompletePersonalityModel
    
    print("✅ Successfully imported model components")
    
    # Initialize feature extractors with GPU optimization
    print("🔧 Initializing feature extractors...")
    
    # Initialize static feature extractor (ResNet-50 based)
    with tf.device('/GPU:0' if physical_gpus else '/CPU:0'):
        static_extractor = StaticFeatureExtractor()
        print("✅ Static feature extractor initialized (ResNet-50)")
    
    # Initialize dynamic feature extractor (I3D based)
    with tf.device('/GPU:0' if physical_gpus else '/CPU:0'):
        dynamic_extractor = DynamicFeatureExtractor()
        print("✅ Dynamic feature extractor initialized (I3D)")
    
    # Model configuration
    model_config = {
        'static_dim': 512,      # ResNet-50 output dimension
        'dynamic_dim': 256,     # I3D output dimension
        'fusion_dim': 128,      # Cross-attention fusion dimension
        'num_heads': 8,         # Multi-head attention heads
        'dropout_rate': 0.3,    # Dropout for regularization
        'learning_rate': 0.001, # Initial learning rate
        'batch_size': 32,       # Training batch size
        'epochs': 50,           # Training epochs
        'validation_split': 0.2 # Validation data percentage
    }
    
    print(f"\n📋 Model Configuration:")
    for key, value in model_config.items():
        print(f"   {key}: {value}")
    
    # Save model configuration
    with open(os.path.join(results_dir, 'model_config.json'), 'w') as f:
        json.dump(model_config, f, indent=2)
    
except ImportError as e:
    print(f"❌ Error importing model components: {e}")
    print("   Please ensure all model files are properly configured")
    
except Exception as e:
    print(f"❌ Error initializing models: {e}")

In [ ]:
# GPU-Optimized Static Feature Extraction Functions
print("🎯 GPU-Optimized Feature Extraction Functions")
print("=" * 50)

def extract_static_features_batch_gpu(face_image_paths, batch_size=32, target_size=(224, 224)):
    """
    Extract static features from face images using GPU-optimized batch processing
    
    Args:
        face_image_paths: List of paths to face images
        batch_size: Number of images to process in each batch
        target_size: Target image size for ResNet-50
    
    Returns:
        numpy array of extracted features [N, 512]
    """
    features = []
    
    with tf.device('/GPU:0' if physical_gpus else '/CPU:0'):
        for i in tqdm(range(0, len(face_image_paths), batch_size), desc="Static features"):
            batch_paths = face_image_paths[i:i+batch_size]
            batch_images = []
            
            # Load and preprocess batch of images
            for img_path in batch_paths:
                try:
                    # Read and preprocess image
                    img = cv2.imread(img_path)
                    if img is not None:
                        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                        img = cv2.resize(img, target_size)
                        img = img.astype(np.float32) / 255.0  # Normalize to [0,1]
                        batch_images.append(img)
                    else:
                        print(f"⚠️ Failed to load image: {img_path}")
                        # Add zero padding for failed images
                        batch_images.append(np.zeros((*target_size, 3), dtype=np.float32))
                except Exception as e:
                    print(f"⚠️ Error processing {img_path}: {e}")
                    batch_images.append(np.zeros((*target_size, 3), dtype=np.float32))
            
            if batch_images:
                # Convert to tensor and extract features
                batch_tensor = tf.convert_to_tensor(batch_images)
                
                # Extract features using static extractor
                try:
                    batch_features = static_extractor.model(batch_tensor)
                    features.extend(batch_features.numpy())
                except Exception as e:
                    print(f"⚠️ Error extracting features for batch: {e}")
                    # Add zero features for failed batch
                    features.extend([np.zeros(512) for _ in batch_images])
            
            # Memory cleanup
            del batch_images, batch_tensor
            gc.collect()
    
    return np.array(features)

def extract_dynamic_features_batch_gpu(optical_flow_paths, batch_size=16, sequence_length=16):
    """
    Extract dynamic features from optical flow sequences using GPU-optimized batch processing
    
    Args:
        optical_flow_paths: List of paths to optical flow .npy files
        batch_size: Number of sequences to process in each batch
        sequence_length: Number of flow frames per sequence
    
    Returns:
        numpy array of extracted features [N, 256]
    """
    features = []
    
    with tf.device('/GPU:0' if physical_gpus else '/CPU:0'):
        for i in tqdm(range(0, len(optical_flow_paths), batch_size), desc="Dynamic features"):
            batch_paths = optical_flow_paths[i:i+batch_size]
            batch_sequences = []
            
            # Load and preprocess batch of optical flow sequences
            for flow_path in batch_paths:
                try:
                    # Load optical flow data
                    flow_data = np.load(flow_path)
                    
                    # Ensure proper shape and normalize
                    if len(flow_data.shape) == 3:  # (H, W, 2)
                        # Resize if needed
                        if flow_data.shape[:2] != (224, 224):
                            flow_data = cv2.resize(flow_data, (224, 224))
                        
                        # Normalize flow magnitude
                        flow_magnitude = np.sqrt(flow_data[:,:,0]**2 + flow_data[:,:,1]**2)
                        max_magnitude = np.max(flow_magnitude) if np.max(flow_magnitude) > 0 else 1.0
                        flow_data = flow_data / max_magnitude
                        
                        batch_sequences.append(flow_data)
                    else:
                        print(f"⚠️ Invalid flow shape {flow_data.shape} for {flow_path}")
                        batch_sequences.append(np.zeros((224, 224, 2), dtype=np.float32))
                        
                except Exception as e:
                    print(f"⚠️ Error loading {flow_path}: {e}")
                    batch_sequences.append(np.zeros((224, 224, 2), dtype=np.float32))
            
            if batch_sequences:
                # Convert to tensor and extract features
                try:
                    batch_tensor = tf.convert_to_tensor(batch_sequences)
                    # Add temporal dimension for 3D CNN if needed
                    if len(batch_tensor.shape) == 4:  # (B, H, W, 2)
                        batch_tensor = tf.expand_dims(batch_tensor, axis=1)  # (B, 1, H, W, 2)
                    
                    # Extract features using dynamic extractor
                    batch_features = dynamic_extractor.model(batch_tensor)
                    features.extend(batch_features.numpy())
                except Exception as e:
                    print(f"⚠️ Error extracting features for batch: {e}")
                    features.extend([np.zeros(256) for _ in batch_sequences])
            
            # Memory cleanup
            del batch_sequences, batch_tensor
            gc.collect()
    
    return np.array(features)

def load_existing_features(features_path):
    """Load existing features if available for resume functionality"""
    if os.path.exists(features_path):
        try:
            features = np.load(features_path)
            print(f"✅ Loaded existing features: {features.shape}")
            return features
        except Exception as e:
            print(f"⚠️ Error loading features: {e}")
    return None

print("✅ GPU-optimized feature extraction functions defined")
print("   • Batch processing for memory efficiency")
print("   • GPU acceleration with TensorFlow")
print("   • Error handling and resume capability")
print("   • Memory management and cleanup")

In [ ]:
# Static Feature Extraction Pipeline with Resume Capability
print("🎯 Starting GPU-Accelerated Static Feature Extraction")
print("=" * 60)

static_features_path = os.path.join(features_dir, 'static_features_large.npy')
static_extraction_log = os.path.join(results_dir, 'static_feature_extraction_log.json')

# Check for existing features (resume functionality)
existing_static_features = load_existing_features(static_features_path)

if existing_static_features is not None:
    print(f"🔄 Found existing static features: {existing_static_features.shape}")
    print("   Skipping static feature extraction (use resume mode)")
    static_features = existing_static_features
else:
    print("🚀 Starting fresh static feature extraction...")
    
    # Collect all face image paths
    face_image_paths = []
    video_labels_map = {}  # Map video paths to labels for alignment
    
    start_time = time.time()
    
    print("📋 Collecting face image paths...")
    for training_dir in tqdm(os.listdir(faces_base_dir), desc="Scanning directories"):
        training_path = os.path.join(faces_base_dir, training_dir)
        if not os.path.isdir(training_path):
            continue
            
        video_dirs = sorted([d for d in os.listdir(training_path) 
                            if os.path.isdir(os.path.join(training_path, d))])
        
        for video_dir in video_dirs:
            video_path = os.path.join(training_path, video_dir)
            face_files = sorted([f for f in os.listdir(video_path) if f.endswith('.jpg')])
            
            for face_file in face_files:
                face_path = os.path.join(video_path, face_file)
                face_image_paths.append(face_path)
                
                # Store video-to-directory mapping for label alignment
                video_id = f"{training_dir}/{video_dir}"
                video_labels_map[face_path] = video_id
    
    print(f"📊 Collected {len(face_image_paths):,} face image paths")
    
    # Extract static features with GPU acceleration
    print(f"🔥 Extracting static features using GPU acceleration...")
    
    extraction_stats = {
        'start_time': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'total_images': len(face_image_paths),
        'batch_size': model_config['batch_size'],
        'gpu_acceleration': len(physical_gpus) > 0
    }
    
    try:
        static_features = extract_static_features_batch_gpu(
            face_image_paths, 
            batch_size=model_config['batch_size']
        )
        
        # Save extracted features
        np.save(static_features_path, static_features)
        
        end_time = time.time()
        extraction_time = end_time - start_time
        
        extraction_stats.update({
            'end_time': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'extraction_time_seconds': extraction_time,
            'extraction_time_formatted': str(time.strftime('%H:%M:%S', time.gmtime(extraction_time))),
            'features_shape': static_features.shape,
            'features_saved_to': static_features_path,
            'average_time_per_image': extraction_time / len(face_image_paths),
            'images_per_second': len(face_image_paths) / extraction_time
        })
        
        print(f"\n🎉 Static Feature Extraction Completed!")
        print(f"   ⏱️  Processing time: {extraction_stats['extraction_time_formatted']}")
        print(f"   📊 Features extracted: {static_features.shape}")
        print(f"   💾 Saved to: {static_features_path}")
        print(f"   ⚡ Speed: {extraction_stats['images_per_second']:.2f} images/second")
        print(f"   🎯 GPU acceleration: {'✅ Enabled' if extraction_stats['gpu_acceleration'] else '❌ Disabled'}")
        
    except Exception as e:
        print(f"❌ Error during static feature extraction: {e}")
        extraction_stats['error'] = str(e)
        static_features = None
    
    # Save extraction log
    with open(static_extraction_log, 'w') as f:
        json.dump(extraction_stats, f, indent=2)
    
    print(f"📋 Extraction log saved to: {static_extraction_log}")

# Memory cleanup
gc.collect()

In [ ]:
# Dynamic Feature Extraction Pipeline with Resume Capability  
print("🌊 Starting GPU-Accelerated Dynamic Feature Extraction")
print("=" * 60)

dynamic_features_path = os.path.join(features_dir, 'dynamic_features_large.npy')
dynamic_extraction_log = os.path.join(results_dir, 'dynamic_feature_extraction_log.json')

# Check for existing features (resume functionality)
existing_dynamic_features = load_existing_features(dynamic_features_path)

if existing_dynamic_features is not None:
    print(f"🔄 Found existing dynamic features: {existing_dynamic_features.shape}")
    print("   Skipping dynamic feature extraction (use resume mode)")
    dynamic_features = existing_dynamic_features
else:
    print("🚀 Starting fresh dynamic feature extraction...")
    
    # Collect all optical flow paths
    optical_flow_paths = []
    flow_video_map = {}  # Map flow paths to videos for alignment
    
    start_time = time.time()
    
    print("📋 Collecting optical flow paths...")
    for training_dir in tqdm(os.listdir(flow_base_dir), desc="Scanning flow directories"):
        training_path = os.path.join(flow_base_dir, training_dir)
        if not os.path.isdir(training_path):
            continue
            
        video_dirs = sorted([d for d in os.listdir(training_path) 
                            if os.path.isdir(os.path.join(training_path, d))])
        
        for video_dir in video_dirs:
            video_path = os.path.join(training_path, video_dir)
            flow_files = sorted([f for f in os.listdir(video_path) if f.endswith('.npy')])
            
            for flow_file in flow_files:
                flow_path = os.path.join(video_path, flow_file)
                optical_flow_paths.append(flow_path)
                
                # Store flow-to-video mapping for alignment
                video_id = f"{training_dir}/{video_dir}"
                flow_video_map[flow_path] = video_id
    
    print(f"📊 Collected {len(optical_flow_paths):,} optical flow paths")
    
    # Extract dynamic features with GPU acceleration
    print(f"🔥 Extracting dynamic features using GPU acceleration...")
    
    extraction_stats = {
        'start_time': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'total_flows': len(optical_flow_paths),
        'batch_size': model_config['batch_size'] // 2,  # Smaller batch for 3D processing
        'gpu_acceleration': len(physical_gpus) > 0
    }
    
    try:
        dynamic_features = extract_dynamic_features_batch_gpu(
            optical_flow_paths, 
            batch_size=model_config['batch_size'] // 2  # Reduced batch size for memory
        )
        
        # Save extracted features
        np.save(dynamic_features_path, dynamic_features)
        
        end_time = time.time()
        extraction_time = end_time - start_time
        
        extraction_stats.update({
            'end_time': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'extraction_time_seconds': extraction_time,
            'extraction_time_formatted': str(time.strftime('%H:%M:%S', time.gmtime(extraction_time))),
            'features_shape': dynamic_features.shape,
            'features_saved_to': dynamic_features_path,
            'average_time_per_flow': extraction_time / len(optical_flow_paths),
            'flows_per_second': len(optical_flow_paths) / extraction_time
        })
        
        print(f"\n🎉 Dynamic Feature Extraction Completed!")
        print(f"   ⏱️  Processing time: {extraction_stats['extraction_time_formatted']}")
        print(f"   📊 Features extracted: {dynamic_features.shape}")
        print(f"   💾 Saved to: {dynamic_features_path}")
        print(f"   ⚡ Speed: {extraction_stats['flows_per_second']:.2f} flows/second")
        print(f"   🎯 GPU acceleration: {'✅ Enabled' if extraction_stats['gpu_acceleration'] else '❌ Disabled'}")
        
    except Exception as e:
        print(f"❌ Error during dynamic feature extraction: {e}")
        extraction_stats['error'] = str(e)
        dynamic_features = None
    
    # Save extraction log
    with open(dynamic_extraction_log, 'w') as f:
        json.dump(extraction_stats, f, indent=2)
    
    print(f"📋 Extraction log saved to: {dynamic_extraction_log}")

# Memory cleanup
gc.collect()

In [ ]:
# Data Alignment and Label Processing
print("🔗 Data Alignment and Label Processing")
print("=" * 50)

# Load personality trait labels from ChaLearn dataset
def load_personality_labels():
    """
    Load personality trait labels from the ChaLearn dataset structure
    Returns aligned labels for the extracted features
    """
    labels_path = os.path.join('data', 'training_data', 'annotation_training.pkl')
    sample_labels_path = os.path.join('data_sample', 'labels.npy')
    
    # Try to load from multiple sources
    if os.path.exists(sample_labels_path):
        print(f"📁 Loading sample labels from: {sample_labels_path}")
        sample_labels = np.load(sample_labels_path)
        print(f"   Sample labels shape: {sample_labels.shape}")
        return sample_labels
    
    # If no labels found, create dummy labels for testing
    print("⚠️ No personality labels found - creating dummy labels for testing")
    
    # Estimate number of unique videos from features
    num_samples = min(len(static_features), len(dynamic_features)) if (static_features is not None and dynamic_features is not None) else 960
    
    # Create dummy OCEAN labels (5 traits: Openness, Conscientiousness, Extraversion, Agreeableness, Neuroticism)
    dummy_labels = np.random.uniform(0.0, 1.0, (num_samples, 5))
    
    print(f"   Created dummy labels shape: {dummy_labels.shape}")
    print("   ⚠️ These are random labels - replace with actual ChaLearn annotations for real training")
    
    return dummy_labels

# Align features and labels
print("🔄 Aligning features and labels...")

alignment_stats = {
    'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'static_features_available': static_features is not None,
    'dynamic_features_available': dynamic_features is not None,
}

if static_features is not None and dynamic_features is not None:
    # Load labels
    personality_labels = load_personality_labels()
    
    # Calculate alignment requirements
    static_count = len(static_features)
    dynamic_count = len(dynamic_features)
    label_count = len(personality_labels)
    
    alignment_stats.update({
        'static_features_count': static_count,
        'dynamic_features_count': dynamic_count,
        'labels_count': label_count,
        'min_samples': min(static_count, dynamic_count, label_count)
    })
    
    print(f"📊 Alignment Analysis:")
    print(f"   • Static features: {static_count:,}")
    print(f"   • Dynamic features: {dynamic_count:,}")
    print(f"   • Personality labels: {label_count:,}")
    
    # Align to minimum count for consistency
    min_samples = alignment_stats['min_samples']
    
    if min_samples > 0:
        # Truncate to minimum sample count
        static_features_aligned = static_features[:min_samples]
        dynamic_features_aligned = dynamic_features[:min_samples]
        labels_aligned = personality_labels[:min_samples]
        
        # Verify alignment
        assert len(static_features_aligned) == len(dynamic_features_aligned) == len(labels_aligned), \
            "Feature alignment failed!"
        
        print(f"✅ Data alignment successful!")
        print(f"   • Aligned samples: {min_samples:,}")
        print(f"   • Static features shape: {static_features_aligned.shape}")
        print(f"   • Dynamic features shape: {dynamic_features_aligned.shape}")
        print(f"   • Labels shape: {labels_aligned.shape}")
        
        # Save aligned data
        aligned_data_path = os.path.join(features_dir, 'aligned_features_and_labels.npz')
        np.savez_compressed(
            aligned_data_path,
            static_features=static_features_aligned,
            dynamic_features=dynamic_features_aligned,
            labels=labels_aligned
        )
        
        alignment_stats.update({
            'alignment_successful': True,
            'aligned_samples': min_samples,
            'aligned_data_saved_to': aligned_data_path
        })
        
        print(f"💾 Aligned data saved to: {aligned_data_path}")
        
    else:
        print("❌ No samples available for alignment")
        alignment_stats['alignment_successful'] = False
        
else:
    print("❌ Missing feature data - cannot perform alignment")
    alignment_stats['alignment_successful'] = False

# Save alignment results
alignment_log_path = os.path.join(results_dir, 'data_alignment_log.json')
with open(alignment_log_path, 'w') as f:
    json.dump(alignment_stats, f, indent=2)

print(f"📋 Alignment log saved to: {alignment_log_path}")

In [ ]:
# Cross-Attention CNN Model Training Pipeline
print("🚀 Cross-Attention CNN Model Training Pipeline")
print("=" * 60)

if alignment_stats.get('alignment_successful', False):
    print("🔥 Starting model training with aligned data...")
    
    # Load aligned data
    aligned_data_path = os.path.join(features_dir, 'aligned_features_and_labels.npz')
    aligned_data = np.load(aligned_data_path)
    
    X_static = aligned_data['static_features']
    X_dynamic = aligned_data['dynamic_features']
    y = aligned_data['labels']
    
    print(f"📊 Training data loaded:")
    print(f"   • Static features: {X_static.shape}")
    print(f"   • Dynamic features: {X_dynamic.shape}")
    print(f"   • Labels (OCEAN): {y.shape}")
    
    # Data preprocessing
    print("🔧 Preprocessing data...")
    
    # Normalize features
    static_scaler = StandardScaler()
    dynamic_scaler = StandardScaler()
    
    X_static_norm = static_scaler.fit_transform(X_static)
    X_dynamic_norm = dynamic_scaler.fit_transform(X_dynamic)
    
    # Split data
    train_indices, test_indices = train_test_split(
        range(len(X_static_norm)), 
        test_size=0.2, 
        random_state=42,
        stratify=None  # No stratification for regression
    )
    
    X_static_train = X_static_norm[train_indices]
    X_static_test = X_static_norm[test_indices]
    X_dynamic_train = X_dynamic_norm[train_indices]
    X_dynamic_test = X_dynamic_norm[test_indices]
    y_train = y[train_indices]
    y_test = y[test_indices]
    
    print(f"📈 Data split:")
    print(f"   • Training samples: {len(X_static_train)}")
    print(f"   • Testing samples: {len(X_static_test)}")
    
    # Initialize and compile model
    print("🤖 Initializing Cross-Attention CNN model...")
    
    with tf.device('/GPU:0' if physical_gpus else '/CPU:0'):
        model = CompletePersonalityModel(
            static_dim=model_config['static_dim'],
            dynamic_dim=model_config['dynamic_dim'],
            fusion_dim=model_config['fusion_dim'],
            num_heads=model_config['num_heads'],
            dropout_rate=model_config['dropout_rate']
        )
        
        # Compile model
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=model_config['learning_rate']),
            loss='mse',
            metrics=['mae', 'mse']
        )
        
        print("✅ Model compiled successfully")
    
    # Training configuration
    training_config = {
        'epochs': model_config['epochs'],
        'batch_size': model_config['batch_size'],
        'validation_split': 0.15,  # Use validation split from training data
        'early_stopping_patience': 10,
        'reduce_lr_patience': 5,
        'checkpoint_period': 5
    }
    
    # Setup callbacks
    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss', 
            patience=training_config['early_stopping_patience'], 
            restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss', 
            factor=0.5, 
            patience=training_config['reduce_lr_patience'], 
            min_lr=1e-7
        ),
        tf.keras.callbacks.ModelCheckpoint(
            filepath=os.path.join(models_dir, 'cross_attention_cnn_checkpoint.h5'),
            monitor='val_loss',
            save_best_only=True,
            save_weights_only=False,
            period=training_config['checkpoint_period']
        )
    ]
    
    print(f"📋 Training configuration:")
    for key, value in training_config.items():
        print(f"   {key}: {value}")
    
    # Train model
    print(f"\n🔥 Starting model training...")
    training_start_time = time.time()
    
    try:
        history = model.fit(
            [X_static_train, X_dynamic_train], 
            y_train,
            epochs=training_config['epochs'],
            batch_size=training_config['batch_size'],
            validation_split=training_config['validation_split'],
            callbacks=callbacks,
            verbose=1
        )
        
        training_end_time = time.time()
        training_time = training_end_time - training_start_time
        
        print(f"\n🎉 Model training completed!")
        print(f"   ⏱️  Training time: {time.strftime('%H:%M:%S', time.gmtime(training_time))}")
        
        # Evaluate model
        print(f"📊 Evaluating model on test set...")
        
        test_loss, test_mae, test_mse = model.evaluate(
            [X_static_test, X_dynamic_test], 
            y_test, 
            verbose=0
        )
        
        # Make predictions for detailed evaluation
        y_pred = model.predict([X_static_test, X_dynamic_test])
        
        # Calculate additional metrics
        r2_scores = [r2_score(y_test[:, i], y_pred[:, i]) for i in range(5)]
        trait_names = ['Openness', 'Conscientiousness', 'Extraversion', 'Agreeableness', 'Neuroticism']
        
        evaluation_results = {
            'test_loss': float(test_loss),
            'test_mae': float(test_mae),
            'test_mse': float(test_mse),
            'test_rmse': float(np.sqrt(test_mse)),
            'r2_scores': {trait_names[i]: float(r2_scores[i]) for i in range(5)},
            'mean_r2_score': float(np.mean(r2_scores)),
            'training_time_seconds': training_time,
            'training_history': {
                'loss': [float(x) for x in history.history['loss']],
                'val_loss': [float(x) for x in history.history['val_loss']],
                'mae': [float(x) for x in history.history['mae']],
                'val_mae': [float(x) for x in history.history['val_mae']]
            }
        }
        
        print(f"📈 Evaluation Results:")
        print(f"   • Test Loss (MSE): {test_loss:.4f}")
        print(f"   • Test MAE: {test_mae:.4f}")
        print(f"   • Test RMSE: {np.sqrt(test_mse):.4f}")
        print(f"   • Mean R² Score: {np.mean(r2_scores):.4f}")
        print(f"   • Individual R² Scores:")
        for i, trait in enumerate(trait_names):
            print(f"     - {trait}: {r2_scores[i]:.4f}")
        
        # Save model and results
        model_save_path = os.path.join(models_dir, 'cross_attention_cnn_final.h5')
        model.save(model_save_path)
        
        results_save_path = os.path.join(results_dir, 'training_results.json')
        with open(results_save_path, 'w') as f:
            json.dump(evaluation_results, f, indent=2)
        
        print(f"\n💾 Model and results saved:")
        print(f"   • Model: {model_save_path}")
        print(f"   • Results: {results_save_path}")
        
    except Exception as e:
        print(f"❌ Error during model training: {e}")
        
else:
    print("❌ Cannot start training - data alignment failed")
    print("   Please ensure feature extraction completed successfully")

In [ ]:
# Training Visualization and Analysis
print("📊 Training Results Visualization and Analysis")
print("=" * 50)

# Check if training results are available
results_path = os.path.join(results_dir, 'training_results.json')

if os.path.exists(results_path):
    # Load training results
    with open(results_path, 'r') as f:
        training_results = json.load(f)
    
    print("✅ Training results loaded successfully")
    
    # Create visualizations
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('Cross-Attention CNN Training Results', fontsize=16, fontweight='bold')
    
    # Plot 1: Training and Validation Loss
    history = training_results['training_history']
    epochs = range(1, len(history['loss']) + 1)
    
    axes[0, 0].plot(epochs, history['loss'], 'b-', label='Training Loss', linewidth=2)
    axes[0, 0].plot(epochs, history['val_loss'], 'r-', label='Validation Loss', linewidth=2)
    axes[0, 0].set_title('Training and Validation Loss', fontweight='bold')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss (MSE)')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Plot 2: Training and Validation MAE
    axes[0, 1].plot(epochs, history['mae'], 'g-', label='Training MAE', linewidth=2)
    axes[0, 1].plot(epochs, history['val_mae'], 'orange', label='Validation MAE', linewidth=2)
    axes[0, 1].set_title('Training and Validation MAE', fontweight='bold')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Mean Absolute Error')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # Plot 3: R² Scores by Personality Trait
    r2_scores = training_results['r2_scores']
    traits = list(r2_scores.keys())
    scores = list(r2_scores.values())
    
    colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4', '#FFEAA7']
    bars = axes[1, 0].bar(traits, scores, color=colors, alpha=0.8, edgecolor='black', linewidth=1)
    axes[1, 0].set_title('R² Scores by Personality Trait', fontweight='bold')
    axes[1, 0].set_ylabel('R² Score')
    axes[1, 0].tick_params(axis='x', rotation=45)
    axes[1, 0].grid(True, alpha=0.3, axis='y')
    
    # Add value labels on bars
    for bar, score in zip(bars, scores):
        height = bar.get_height()
        axes[1, 0].text(bar.get_x() + bar.get_width()/2., height + 0.01,
                       f'{score:.3f}', ha='center', va='bottom', fontweight='bold')
    
    # Plot 4: Model Performance Summary
    metrics = ['Test Loss', 'Test MAE', 'Test RMSE', 'Mean R²']
    values = [
        training_results['test_loss'],
        training_results['test_mae'],
        training_results['test_rmse'],
        training_results['mean_r2_score']
    ]
    
    # Normalize values for better visualization
    normalized_values = [(v - min(values)) / (max(values) - min(values)) for v in values]
    
    bars = axes[1, 1].bar(metrics, normalized_values, color=['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4'], 
                         alpha=0.8, edgecolor='black', linewidth=1)
    axes[1, 1].set_title('Model Performance Summary (Normalized)', fontweight='bold')
    axes[1, 1].set_ylabel('Normalized Score')
    axes[1, 1].tick_params(axis='x', rotation=45)
    axes[1, 1].grid(True, alpha=0.3, axis='y')
    
    # Add actual value labels
    for bar, value in zip(bars, values):
        height = bar.get_height()
        axes[1, 1].text(bar.get_x() + bar.get_width()/2., height + 0.02,
                       f'{value:.3f}', ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    
    # Save visualization
    viz_save_path = os.path.join(results_dir, 'training_visualization.png')
    plt.savefig(viz_save_path, dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"📊 Visualization saved to: {viz_save_path}")
    
    # Performance summary
    print(f"\n🎯 Final Model Performance Summary:")
    print(f"   🔹 Test Loss (MSE): {training_results['test_loss']:.4f}")
    print(f"   🔹 Test MAE: {training_results['test_mae']:.4f}")
    print(f"   🔹 Test RMSE: {training_results['test_rmse']:.4f}")
    print(f"   🔹 Mean R² Score: {training_results['mean_r2_score']:.4f}")
    print(f"   🔹 Training Time: {time.strftime('%H:%M:%S', time.gmtime(training_results['training_time_seconds']))}")
    
    # Create performance report
    performance_report = f"""
# Cross-Attention CNN Performance Report

## Model Configuration
- Static Feature Dimension: {model_config['static_dim']}
- Dynamic Feature Dimension: {model_config['dynamic_dim']}
- Fusion Dimension: {model_config['fusion_dim']}
- Number of Attention Heads: {model_config['num_heads']}
- Dropout Rate: {model_config['dropout_rate']}

## Training Results
- **Test Loss (MSE)**: {training_results['test_loss']:.4f}
- **Test MAE**: {training_results['test_mae']:.4f}
- **Test RMSE**: {training_results['test_rmse']:.4f}
- **Mean R² Score**: {training_results['mean_r2_score']:.4f}

## Individual Trait Performance (R² Scores)
"""
    
    for trait, score in training_results['r2_scores'].items():
        performance_report += f"- **{trait}**: {score:.4f}\n"
    
    performance_report += f"""
## Training Configuration
- Epochs: {model_config['epochs']}
- Batch Size: {model_config['batch_size']}
- Learning Rate: {model_config['learning_rate']}
- Training Time: {time.strftime('%H:%M:%S', time.gmtime(training_results['training_time_seconds']))}

Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
"""
    
    # Save performance report
    report_path = os.path.join(results_dir, 'performance_report.md')
    with open(report_path, 'w') as f:
        f.write(performance_report)
    
    print(f"📋 Performance report saved to: {report_path}")
    
else:
    print("❌ No training results found")
    print("   Please ensure model training completed successfully")

In [ ]:
# Final Pipeline Summary and Next Steps
print("🏁 Cross-Attention CNN Pipeline Summary")
print("=" * 60)

# Collect all pipeline results
pipeline_summary = {
    'pipeline_completion_time': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'gpu_acceleration_enabled': len(physical_gpus) > 0,
    'components_completed': []
}

# Check component completion status
components = [
    ('GPU Configuration', True),
    ('Data Analysis', os.path.exists(os.path.join(results_dir, 'data_analysis_results.json'))),
    ('Static Feature Extraction', os.path.exists(os.path.join(features_dir, 'static_features_large.npy'))),
    ('Dynamic Feature Extraction', os.path.exists(os.path.join(features_dir, 'dynamic_features_large.npy'))),
    ('Data Alignment', os.path.exists(os.path.join(features_dir, 'aligned_features_and_labels.npz'))),
    ('Model Training', os.path.exists(os.path.join(models_dir, 'cross_attention_cnn_final.h5'))),
    ('Results Visualization', os.path.exists(os.path.join(results_dir, 'training_visualization.png')))
]

print(f"📊 Pipeline Component Status:")
for component_name, status in components:
    status_icon = "✅" if status else "❌"
    print(f"   {status_icon} {component_name}")
    pipeline_summary['components_completed'].append({
        'component': component_name,
        'completed': status
    })

# Calculate overall completion rate
completed_count = sum([1 for _, status in components if status])
completion_rate = (completed_count / len(components)) * 100

pipeline_summary['completion_rate'] = completion_rate
pipeline_summary['completed_components'] = completed_count
pipeline_summary['total_components'] = len(components)

print(f"\n📈 Overall Pipeline Completion: {completion_rate:.1f}% ({completed_count}/{len(components)})")

# Display key outputs
print(f"\n📁 Key Output Files:")
key_outputs = [
    ('Static Features', os.path.join(features_dir, 'static_features_large.npy')),
    ('Dynamic Features', os.path.join(features_dir, 'dynamic_features_large.npy')),
    ('Aligned Data', os.path.join(features_dir, 'aligned_features_and_labels.npz')),
    ('Trained Model', os.path.join(models_dir, 'cross_attention_cnn_final.h5')),
    ('Training Results', os.path.join(results_dir, 'training_results.json')),
    ('Performance Report', os.path.join(results_dir, 'performance_report.md')),
    ('Visualization', os.path.join(results_dir, 'training_visualization.png'))
]

for output_name, output_path in key_outputs:
    exists = os.path.exists(output_path)
    status_icon = "✅" if exists else "❌"
    size_info = ""
    if exists:
        try:
            size_bytes = os.path.getsize(output_path)
            if size_bytes > 1024*1024:  # > 1MB
                size_info = f" ({size_bytes/(1024*1024):.1f} MB)"
            else:
                size_info = f" ({size_bytes/1024:.1f} KB)"
        except:
            pass
    print(f"   {status_icon} {output_name}: {output_path}{size_info}")

# Performance summary (if available)
if os.path.exists(os.path.join(results_dir, 'training_results.json')):
    try:
        with open(os.path.join(results_dir, 'training_results.json'), 'r') as f:
            training_results = json.load(f)
        
        pipeline_summary['model_performance'] = {
            'test_mae': training_results['test_mae'],
            'test_rmse': training_results['test_rmse'],
            'mean_r2_score': training_results['mean_r2_score']
        }
        
        print(f"\n🎯 Model Performance:")
        print(f"   • Test MAE: {training_results['test_mae']:.4f}")
        print(f"   • Test RMSE: {training_results['test_rmse']:.4f}")
        print(f"   • Mean R² Score: {training_results['mean_r2_score']:.4f}")
        
    except Exception as e:
        print(f"⚠️ Could not load performance metrics: {e}")

# Next steps and recommendations
print(f"\n🎯 Next Steps and Recommendations:")

if completion_rate == 100:
    print(f"   🎉 CONGRATULATIONS! Pipeline completed successfully!")
    print(f"   ✅ All components have been executed")
    print(f"   📊 Model is trained and ready for deployment")
    print(f"   🔍 Review performance metrics and consider hyperparameter tuning")
    print(f"   📝 Consider running ablation studies to analyze component contributions")
    print(f"   🚀 Ready for Phase 5: Model evaluation and research analysis")
else:
    print(f"   ⚠️ Pipeline incomplete ({completion_rate:.1f}% complete)")
    failed_components = [name for name, status in components if not status]
    print(f"   🔧 Address failed components: {', '.join(failed_components)}")
    print(f"   🔄 Use resume functionality to continue from last checkpoint")

# Save pipeline summary
summary_path = os.path.join(results_dir, 'pipeline_summary.json')
with open(summary_path, 'w') as f:
    json.dump(pipeline_summary, f, indent=2)

print(f"\n💾 Pipeline summary saved to: {summary_path}")

# Memory cleanup
gc.collect()

print(f"\n🎊 Cross-Attention CNN Feature Extraction and Training Pipeline Complete!")
print(f"    Thank you for using the GPU-accelerated pipeline!")

In [ ]:
# Test Cell - Verify Package Imports
print("Testing package imports...")
try:
    import numpy as np
    import tensorflow as tf
    import sklearn
    import seaborn as sns
    import h5py
    print("✅ All packages imported successfully!")
    print(f"NumPy: {np.__version__}")
    print(f"TensorFlow: {tf.__version__}")
    print(f"scikit-learn: {sklearn.__version__}")
except ImportError as e:
    print(f"❌ Import error: {e}")